## Описание данных и задание SQL

1. Таблица books

Содержит данные о книгах:
 - book_id — идентификатор книги;
 - author_id — идентификатор автора;
 - title — название книги;
 - num_pages — количество страниц;
 - publication_date — дата публикации книги;
 - publisher_id — идентификатор издателя.
 
2. Таблица authors

Содержит данные об авторах:
 - author_id — идентификатор автора;
 - author — имя автора.
 - Таблица publishers
 - Содержит данные об издательствах:
 - publisher_id — идентификатор издательства;
 - publisher — название издательства;

3. Таблица ratings

Содержит данные о пользовательских оценках книг:
 - rating_id — идентификатор оценки;
 - book_id — идентификатор книги;
 - username — имя пользователя, оставившего оценку;
 - rating — оценка книги.
 
4. Таблица reviews

Содержит данные о пользовательских обзорах на книги:
 - review_id — идентификатор обзора;
 - book_id — идентификатор книги;
 - username — имя пользователя, написавшего обзор;
 - text — текст обзора.

## Задания
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
6. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student',                        # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp',                            # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,                                       # порт подключения
             'db': 'data-analyst-final-project-db'}              # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# создадим функцию, которая будет принимать переменную с sql-запросом и выводить результат запроса
def result(query):
    return pd.io.sql.read_sql(query, con = engine)

## Обзор данных

In [4]:
# посмотрим 5 первых строк таблицы books
query = '''
            SELECT * 
            FROM books
            LIMIT 5
        '''
result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''
            SELECT * 
            FROM authors
            LIMIT 5
        '''
result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''
            SELECT * 
            FROM publishers
            LIMIT 5
        '''
result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = '''
            SELECT * 
            FROM ratings
            LIMIT 5
        '''
result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = '''
            SELECT * 
            FROM reviews
            LIMIT 5
        '''
result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитайте, сколько книг вышло после 1 января 2000 года;

In [9]:
query = '''
            SELECT COUNT(publication_date)
            FROM books
            WHERE publication_date > '2000-01-01'
        '''
result(query)

,count
0,819


После 1 января 2000 года вышло 819 книг.

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px"><b>Excelent work - ✅</b><br> 👍</div> <br>

## Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [10]:
query = '''
            SELECT books.book_id,
                   books.title,
                   COUNT(DISTINCT reviews.review_id) AS cnt_reviews,
                   ROUND(AVG(ratings.rating),2) AS avg_rating
                   
            FROM books
            
            LEFT JOIN reviews AS reviews ON books.book_id=reviews.book_id
            LEFT JOIN ratings AS ratings ON books.book_id=ratings.book_id
            
            GROUP BY books.book_id
            ORDER BY cnt_reviews DESC 
            
        '''
result(query)


,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Максимальное количество обзоров по книге Twilight (Twilight #1) - 7 cо средним рейтингом 3.66.

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px"><b>Excelent work - ✅</b><br> 👍</div> <br>

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [11]:
query = '''
            SELECT books.publisher_id,
                   publishers.publisher,
                   COUNT(books.book_id) AS book_count
                   
            FROM books
            
            LEFT JOIN publishers AS publishers ON books.publisher_id=publishers.publisher_id
            
            WHERE books.num_pages > 50
            GROUP BY books.publisher_id, publishers.publisher_id
            ORDER BY book_count DESC
            
            LIMIT 1
            
        '''
result(query)

,publisher_id,publisher,book_count
0,212,Penguin Books,42


Наибольшее количество книг 42 - выпустило издательство Penguin Books

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px"><b>Excelent work - ✅</b><br> 👍</div> <br>

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [12]:
	
query = '''
            SELECT authors.author,
                   ROUND(AVG(ratings.rating),2) AS avg_rating,
                   COUNT(ratings.rating) AS cnt_rating
                   
            
            FROM ratings
            
            LEFT JOIN books AS books ON ratings.book_id=books.book_id
            LEFT JOIN authors AS authors ON books.author_id=authors.author_id
            
            WHERE ratings.book_id IN ( 
                                       SELECT ratings.book_id
                                       FROM ratings
                                       GROUP BY ratings.book_id
                                       HAVING COUNT(ratings.book_id) >= 50
                                     )
            GROUP BY authors.author
            ORDER BY avg_rating DESC
            
            LIMIT 1
            
        '''
result(query)

,author,avg_rating,cnt_rating
0,J.K. Rowling/Mary GrandPré,4.29,310


Автор с высокой средней оценкой книг J.K. Rowling/Mary GrandPré

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px"><b>Excelent work - ✅</b><br> 👍</div> <br>

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [13]:
query = '''
            SELECT ROUND(AVG(review_cnt),2) AS avg_cnt_review       
            
            FROM (
                   SELECT username,
                   COUNT(review_id) AS review_cnt
                   
                   FROM reviews
                   
                   GROUP BY username
                   HAVING username IN (
                                        SELECT username
                                        FROM ratings
                                        GROUP BY username
                                        HAVING COUNT(rating_id) > 48   
                                      ) 
                 ) AS rew
            
        '''
result(query)

,avg_cnt_review
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px"><b>Excelent work - ✅</b><br> 👍</div> <br>

<img src=https://kto-ozvuchival.ru/wp-content/uploads/2017/12/kto-ozvuchivaet-toni-starka-na-russkom-800x500.jpg><br><hr style="border: 10px solid green;" />

## <span style="color:green">Финальный комментарий<span>

Спасибо за проделанную работу🤝

Ялкун, молодец, проект принят👌
    
Желаю дальнейших успехов и отличного настроения!
    

Допматериалы:
- из общего:
  - для прокачки <code>SQL</code> и большего понимания как всё устроено, рекомендую <a href="https://www.youtube.com/playlist?list=PLkcP_moW_BpOs4gO6SrPrvXu0sPcTyUyp">youtube лекции</a>
  - Для более приятного и удобного оформления SQL запросов рекомендую <a href="https://zelark.github.io/8-tips-to-make-your-sql-more-readable/">статью</a> к прочтению.
- тренажеры:
  - [SQL-ex](https://www.sql-ex.ru/?Lang=0) - Адская вещь, но заставляет хорошо думать над своим запросом
  - [hackerrank](https://www.hackerrank.com/domains/sql)
  - Одной из моих коллег нравится [khanacademy](https://www.khanacademy.org/computing/computer-programming/sql)
  - и мой любимый способ: самостоятельная практика. Установить локально postgres\sqlite, найти любые интересные данные, условно, курсы валют, погоду, данные о продажах (например, известный [sample-superstore](https://community.tableau.com/s/question/0D54T00000CWeX8SAL/sample-superstore-sales-excelxls)) и практиковаться, придумывая себе задачи. А если есть интересующая тебя тема + самостоятельная практика - это лучшее решение
  - оптимизация:
      - [Основы Explain](https://habr.com/ru/articles/203320/)
      - [Разбор Explain](https://habr.com/ru/companies/tensor/articles/505348/) + [Рецепты оптимизации](https://habr.com/ru/companies/tensor/articles/492694/)
      - начать можно с [этого](https://t.me/double_data/52?single) - НЕиспользование преобразований в WHERE
      - общая информация об использовании [DISTINCT](https://jmarquesdatabeyond.medium.com/sql-like-a-pro-please-stop-using-distinct-31bdb6481256)
- стоит ознакомится с такой субд как clickhouse:
 - [Материалы](https://clickhouse.com/docs/ru) из офф документации
 - 💰[Course by Stepik](https://stepik.org/course/100210/syllabus)


Поздравляю, проект принят👍